In [17]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, Comment
import requests
import re
import html5lib
import lxml
import torch
import torch.nn as nn
from torch import Tensor
from tqdm import tqdm,trange
from torch.utils.data import Dataset
from tqdm import tqdm,trange
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
from data_helpers.scrape_utils import *
from data_helpers.data_prep import *
from modeling.pytorch_bball import *

In [37]:
year_list = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

advanced_stats = advanced_stats_crawl(year_list)
advanced_stats.head()

Starting: 2010
Starting: 2011
Starting: 2012
Starting: 2013
Starting: 2014
Starting: 2015
Starting: 2016
Starting: 2017
Starting: 2018
Starting: 2019


,player,pos,age,team,games,mp,per,ts,3par,ftr,...,ows,dws,ws,ws48,obpm,dbpm,bpm,vorp,year,player_join
0,Arron Afflalo,SG,24,DEN,82,2221,10.9,.576,.426,.168,...,2.8,1.4,4.3,.092,0.8,-0.5,0.3,1.3,2010,A. Afflalo
1,Alexis Ajinça,C,21,CHA,6,30,6.3,.479,.000,.100,...,-0.1,0.0,0.0,-0.013,-8.5,-2.0,-10.4,-0.1,2010,A. Ajinça
2,LaMarcus Aldridge,PF,24,POR,78,2922,18.2,.535,.014,.260,...,5.5,3.3,8.8,.145,0.9,0.6,1.6,2.6,2010,L. Aldridge
3,Joe Alexander,SF,23,CHI,8,29,2.8,.273,.167,.500,...,0.0,0.0,0.0,.030,-5.2,1.5,-3.7,0.0,2010,J. Alexander
4,Malik Allen,PF,31,DEN,51,456,5.9,.431,.052,.112,...,-0.3,0.3,0.1,.009,-4.2,-0.8,-5.0,-0.3,2010,M. Allen


In [38]:
## Get roster information

## Have to go team by team

team_list = ['TOR', 'BOS', 'DET', 'CHO', 'PHO', 'BRK', 'NYK', 'LAL', 'LAC',
            'MEM', 'DEN', 'WAS', 'HOU', 'GSW', 'PHI', 'UTA', 'MIL', 'IND',
            'CHI', 'CLE', 'SAC', 'POR', 'OKC', 'MIN', 'SAS', 'DAL', 'NOP',
            'MIA', 'ORL', 'ATL']

roster_info = roster_crawl(team_list, year_list)
roster_info

Starting: 2010
Starting: 2011
Starting: 2012
Starting: 2013
Starting: 2014
Starting: 2015
Starting: 2016
Starting: 2017
Starting: 2018
Starting: 2019


,year,player,position,height,weight,bday,country,yrpro
0,2010,Marcus Banks,PG,74,200,"November 19, 1981",us,6
1,2010,Andrea Bargnani,PF,84,245,"October 26, 1985",it,3
2,2010,Marco Belinelli,SG,77,210,"March 25, 1986",it,2
3,2010,Chris Bosh,C,83,235,"March 24, 1984",us,6
4,2010,José Calderón,PG,75,200,"September 28, 1981",es,4
...,...,...,...,...,...,...,...,...
5285,2019,Jordan Sibert,SG,76,187,"August 1, 1992",us,0
5286,2019,Omari Spellman,PF,81,245,"July 21, 1997",us,0
5287,2019,Taurean Waller-Prince,SF,80,220,"March 22, 1994",us,2
5288,2019,Trae Young,PG,74,180,"September 19, 1998",us,0


In [3]:
'''
## Attempt to get text data - did not work, data not consistently available

players = roster_info['player'].unique()
all_desc = []
all_player = []
for player in players:
    desc_list = []
    player_first = str(player.split()[0])
    player_last = str(player.split()[1])
    print('https://www.nbadraft.net/players/' + player_first + '-' + player_last)
    try:
        page = requests.get('https://www.nbadraft.net/players/' + player_crawl[0] + '-' + player_crawl[1])
        soup = BeautifulSoup(page.content, 'html.parser')
        ret = soup.find_all('class', id='nbap_content_bottom') 
        descriptions = soup.find('div', attrs={'id' : 'nbap_content_bottom'})
        for val in [0,1]:
            alldesc = descriptions.find_all('p')
            desc_list.append(alldesc[val].text)
    except:
        print('not found')
        all_desc.append([])
        all_player.append(player)
    descfin = ' '.join(desc_list)
    all_player.append(player)
    all_desc.append(descfin)
    
'''


"\n## Attempt to get text data - did not work, data not consistently available\n\nplayers = roster_info['player'].unique()\nall_desc = []\nall_player = []\nfor player in players:\n    desc_list = []\n    player_first = str(player.split()[0])\n    player_last = str(player.split()[1])\n    print('https://www.nbadraft.net/players/' + player_first + '-' + player_last)\n    try:\n        page = requests.get('https://www.nbadraft.net/players/' + player_crawl[0] + '-' + player_crawl[1])\n        soup = BeautifulSoup(page.content, 'html.parser')\n        ret = soup.find_all('class', id='nbap_content_bottom') \n        descriptions = soup.find('div', attrs={'id' : 'nbap_content_bottom'})\n        for val in [0,1]:\n            alldesc = descriptions.find_all('p')\n            desc_list.append(alldesc[val].text)\n    except:\n        print('not found')\n        all_desc.append([])\n        all_player.append(player)\n    descfin = ' '.join(desc_list)\n    all_player.append(player)\n    all_desc.a

In [5]:
offsets = ['0', '100', '200', '300', '400', '500', '600', '700', '800', '900', '1000',
          '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000']
    
twoman_data = twoman_crawl(year_list, '200', offsets)

Starting 2010
offset = 0
offset = 100
offset = 200
offset = 300
offset = 400
offset = 500
offset = 600
offset = 700
offset = 800
offset = 900
offset = 1000
offset = 1100
offset = 1200
offset = 1300
offset = 1400
offset = 1500
offset = 1600
offset = 1700
offset = 1800
offset = 1900
offset = 2000
Starting 2011
offset = 0
offset = 100
offset = 200
offset = 300
offset = 400
offset = 500
offset = 600
offset = 700
offset = 800
offset = 900
offset = 1000
offset = 1100
offset = 1200
offset = 1300
offset = 1400
offset = 1500
offset = 1600
offset = 1700
offset = 1800
offset = 1900
offset = 2000
Starting 2012
offset = 0
offset = 100
offset = 200
offset = 300
offset = 400
offset = 500
offset = 600
offset = 700
offset = 800
offset = 900
offset = 1000
offset = 1100
offset = 1200
offset = 1300
offset = 1400
offset = 1500
offset = 1600
offset = 1700
offset = 1800
offset = 1900
offset = 2000
Starting 2013
offset = 0
offset = 100
offset = 200
offset = 300
offset = 400
offset = 500
offset = 600
offset = 

In [7]:
## Write to CSV to avoid going to source each time

twoman_data.to_csv('/Users/Wesley/twoman_netrating/twoman_data.csv', index = False)
roster_info.to_csv('/Users/Wesley/twoman_netrating/roster_data.csv', index = False)
advanced_stats.to_csv('/Users/Wesley/twoman_netrating/advanced_data.csv', index = False)

In [4]:
## Read CSV START HERE

roster_df = pd.read_csv('/Users/Wesley/twoman_netrating/roster_data.csv')
twoman_df = pd.read_csv('/Users/Wesley/twoman_netrating/twoman_data.csv')
advanced_df = pd.read_csv('/Users/Wesley/twoman_netrating/advanced_data.csv')

In [5]:
pd.set_option('display.max_columns', 500)
joined_df = dataset_joiner(twoman_df, roster_df, advanced_df)
joined_df.head()

,team,games_base,minutes,tm_poss,opp_poss,pace,fgm,fga,fg_perc,threep,threepa,threepperc,efg,ftm,fta,ftperc,pts,year,player_left,pos_left,age_left,mp_left,per_left,ts_left,3par_left,ftr_left,orb_left,drb_left,trb_left,ast_left,stl_left,blk_left,tov_left,usg_left,ows_left,dws_left,ws_left,ws48_left,obpm_left,dbpm_left,bpm_left,vorp_left,position_left,height_left,weight_left,country_left,yrpro_left,player_right,pos_right,age_right,mp_right,per_right,ts_right,3par_right,ftr_right,orb_right,drb_right,trb_right,ast_right,stl_right,blk_right,tov_right,usg_right,ows_right,dws_right,ws_right,ws48_right,obpm_right,dbpm_right,bpm_right,vorp_right,position_right,height_right,weight_right,country_right,yrpro_right,team_year
0,UTA,31,203.8,384,387,90.8,13.0,-0.6,0.151,0.8,-7.6,0.186,0.156,-0.1,3.9,-0.085,26.8,2010,Kyrylo Fesenko,C,23.0,408.0,9.7,0.533,0.011,0.600,9.8,16.1,13.1,5.3,0.8,3.7,16.7,15.8,0.1,0.5,0.6,0.070,-3.6,0.6,-2.9,-0.1,C,85.0,288.0,ua,2.0,Deron Williams,PG,25.0,2802.0,20.6,0.574,0.246,0.397,2.2,10.4,6.4,44.5,1.8,0.5,16.9,23.8,7.4,3.0,10.3,0.177,4.3,-1.0,3.3,3.7,PG,75.0,200.0,us,4.0,UTA_2010
1,CLE,75,928.6,1741,1741,90.0,6.2,0.6,0.070,2.8,2.7,0.090,0.086,5.3,7.6,-0.013,20.5,2010,Anthony Parker,SG,34.0,2289.0,9.9,0.576,0.549,0.200,1.6,10.0,6.0,9.9,1.4,0.6,12.1,12.0,2.3,2.6,4.9,0.103,0.2,0.6,0.8,1.6,SG,78.0,215.0,us,6.0,Anderson Varejão,PF,27.0,2166.0,15.8,0.598,0.011,0.375,10.9,20.4,15.9,6.1,1.7,2.4,11.0,13.1,4.3,3.8,8.1,0.179,0.4,2.6,3.0,2.7,PF,82.0,273.0,br,5.0,CLE_2010
2,UTA,33,210.5,400,396,90.7,10.7,-2.9,0.142,-0.3,-7.4,0.127,0.141,-1.5,4.2,-0.125,19.5,2010,Kyrylo Fesenko,C,23.0,408.0,9.7,0.533,0.011,0.600,9.8,16.1,13.1,5.3,0.8,3.7,16.7,15.8,0.1,0.5,0.6,0.070,-3.6,0.6,-2.9,-0.1,C,85.0,288.0,ua,2.0,Wesley Matthews,SG,23.0,2025.0,12.3,0.592,0.292,0.342,3.0,8.0,5.6,9.0,1.6,0.6,12.5,16.5,2.6,1.9,4.6,0.108,0.3,-0.5,-0.2,0.9,SG,77.0,220.0,us,0.0,UTA_2010
3,ORL,55,495.3,937,926,90.3,4.2,-4.5,0.073,6.5,12.8,0.050,0.114,4.3,6.7,-0.021,19.3,2010,Ryan Anderson,PF,21.0,911.0,18.1,0.574,0.544,0.211,9.2,16.2,12.8,7.1,1.4,1.1,11.7,24.4,1.7,1.4,3.1,0.161,2.3,-1.0,1.3,0.8,PF,82.0,240.0,us,1.0,Dwight Howard,C,24.0,2843.0,24.0,0.630,0.008,0.978,12.0,31.3,22.0,8.7,1.4,6.0,18.7,23.9,6.1,7.1,13.2,0.223,0.4,5.0,5.5,5.4,C,83.0,265.0,us,5.0,ORL_2010
4,ORL,21,204.4,385,385,90.4,2.3,-11.9,0.090,3.9,5.5,0.068,0.121,10.6,15.1,-0.021,19.2,2010,Dwight Howard,C,24.0,2843.0,24.0,0.630,0.008,0.978,12.0,31.3,22.0,8.7,1.4,6.0,18.7,23.9,6.1,7.1,13.2,0.223,0.4,5.0,5.5,5.4,C,83.0,265.0,us,5.0,Anthony Johnson,PG,35.0,406.0,12.4,0.543,0.351,0.180,2.4,11.0,6.8,24.2,1.4,0.2,17.8,16.7,0.5,0.5,1.0,0.116,-0.6,-0.2,-0.8,0.1,PG,75.0,190.0,us,12.0,ORL_2010


In [6]:
print('age_left ' + str(len(joined_df['age_left'].unique())/2))
print('position_left ' + str(len(joined_df['position_left'].unique())/2))
print('height_left ' + str(len(joined_df['height_left'].unique())/2))
print('country_left ' + str(len(joined_df['country_left'].unique())/2))
print('yrpro_left ' + str(len(joined_df['yrpro_left'].unique())/2))
print('age_right ' + str(len(joined_df['age_right'].unique())/2))
print('position_right ' + str(len(joined_df['position_right'].unique())/2))
print('height_right ' + str(len(joined_df['height_right'].unique())/2))
print('country_right ' + str(len(joined_df['country_right'].unique())/2))
print('yrpro_right ' + str(len(joined_df['yrpro_right'].unique())/2))

age_left 12.0
position_left 2.5
height_left 10.0
country_left 28.5
yrpro_left 10.5
age_right 12.0
position_right 2.5
height_right 10.0
country_right 27.5
yrpro_right 10.5


In [7]:
modeling_dataset = joined_df[['player_left', 'player_right','team_year', 'year', 'age_left', 'per_left', 'ts_left', '3par_left', 'ftr_left',
                                  'orb_left', 'drb_left', 'trb_left', 'ast_left', 'stl_left', 'blk_left', 'tov_left',
                                  'usg_left', 'ows_left', 'dws_left', 'ws_left', 'ws48_left', 'obpm_left', 'dbpm_left',
                                  'bpm_left', 'vorp_left', 'position_left', 'height_left',
                                  'country_left', 'yrpro_left', 'age_right', 'per_right', 'ts_right', '3par_right', 'ftr_right',
                                  'orb_right', 'drb_right', 'trb_right', 'ast_right', 'stl_right', 'blk_right', 'tov_right',
                                  'usg_right', 'ows_right', 'dws_right', 'ws_right', 'ws48_right', 'obpm_right', 'dbpm_right',
                                  'bpm_right', 'vorp_right', 'position_right', 'height_right', 
                                  'country_right', 'yrpro_right', 'pts']]
cat_features_emb = [('team_year', 50), ('age_left', 12), ('position_left', 5),
                    ('height_left', 10), ('country_left', 30), ('yrpro_left', 12),
                    ('age_right', 12), ('position_right', 5), ('height_right', 10),
                    ('country_right', 30), ('yrpro_right', 12)]

cat_features = [emb[0] for emb in cat_features_emb]
print(cat_features)
cont_features = list(set(modeling_dataset.columns) - set(cat_features))
cont_features.remove('pts')
cont_features.remove('player_left')
cont_features.remove('player_right')
cont_features.remove('year')
print(cont_features)
deep_net_cols = cat_features + cont_features
print(deep_net_cols)

['team_year', 'age_left', 'position_left', 'height_left', 'country_left', 'yrpro_left', 'age_right', 'position_right', 'height_right', 'country_right', 'yrpro_right']
['blk_right', 'obpm_left', 'ows_right', 'dbpm_right', 'dbpm_left', 'orb_left', 'ts_right', 'orb_right', 'tov_right', 'trb_right', '3par_left', 'ws48_right', 'obpm_right', 'trb_left', 'tov_left', 'ws_left', 'usg_right', 'bpm_left', 'ts_left', 'ows_left', 'per_left', 'ws_right', 'blk_left', 'bpm_right', 'ast_left', 'dws_right', 'usg_left', 'vorp_left', 'per_right', 'stl_right', '3par_right', 'ftr_right', 'vorp_right', 'stl_left', 'drb_right', 'ast_right', 'dws_left', 'ftr_left', 'ws48_left', 'drb_left']
['team_year', 'age_left', 'position_left', 'height_left', 'country_left', 'yrpro_left', 'age_right', 'position_right', 'height_right', 'country_right', 'yrpro_right', 'blk_right', 'obpm_left', 'ows_right', 'dbpm_right', 'dbpm_left', 'orb_left', 'ts_right', 'orb_right', 'tov_right', 'trb_right', '3par_left', 'ws48_right', 'ob

In [8]:
train, trainplayers, valid, validplayers, test, testplayers = train_valid_test_splitter(modeling_dataset, 0.8, 305)
train_cats, encoding = cat_encoder(train, cat_features)
valid_cats, encoding = cat_encoder(valid, cat_features, encoding)
test_cats, encoding = cat_encoder(test, cat_features, encoding)

train_base = train_cats.copy()

train_fin, train_y = xandy(min_max(train_cats, train_base, cont_features), 'pts')
valid_fin, valid_y = xandy(min_max(valid_cats, train_base, cont_features), 'pts')
test_fin, test_y = xandy(min_max(test_cats, train_base, cont_features), 'pts')

test.head()

9628
1204
1204
doing this
go
go
41.0
ml
go


,player_left,player_right,team_year,year,age_left,per_left,ts_left,3par_left,ftr_left,orb_left,drb_left,trb_left,ast_left,stl_left,blk_left,tov_left,usg_left,ows_left,dws_left,ws_left,ws48_left,obpm_left,dbpm_left,bpm_left,vorp_left,position_left,height_left,country_left,yrpro_left,age_right,per_right,ts_right,3par_right,ftr_right,orb_right,drb_right,trb_right,ast_right,stl_right,blk_right,tov_right,usg_right,ows_right,dws_right,ws_right,ws48_right,obpm_right,dbpm_right,bpm_right,vorp_right,position_right,height_right,country_right,yrpro_right,pts
2052,Nikola Mirotić,Derrick Rose,75,2015,9,0.530822,0.564987,0.549234,0.362802,0.211823,0.493036,0.446281,0.176471,0.363636,0.24,0.238095,0.519022,0.348066,0.390244,0.364486,0.588689,0.397849,0.431193,0.459330,0.267176,2,1,43,2,0,0.462329,0.387597,0.355580,0.183757,0.128079,0.128134,0.132231,0.532513,0.230769,0.08,0.366071,0.715100,0.182320,0.223684,0.154206,0.300000,0.451456,0.196721,0.318548,0.171233,3,4,0,3,-1.6
2053,Roy Hibbert,Luis Scola,100,2015,6,0.445205,0.419098,0.003282,0.222686,0.443350,0.495822,0.537190,0.132827,0.090909,0.51,0.273810,0.478261,0.237569,0.451220,0.294393,0.434447,0.166667,0.623853,0.354067,0.206107,3,14,0,11,5,0.493151,0.444444,0.033917,0.288761,0.458128,0.598886,0.619835,0.186292,0.307692,0.08,0.258929,0.461538,0.276243,0.421053,0.303738,0.514634,0.325243,0.442623,0.342742,0.184932,4,3,5,0,-1.6
7,Andrei Kirilenko,Kyle Korver,197,2010,6,0.541096,0.649867,0.202407,0.387823,0.270936,0.245125,0.280992,0.258065,0.545455,0.34,0.276786,0.380435,0.364641,0.390244,0.378505,0.604113,0.419355,0.697248,0.617225,0.381679,1,3,47,12,1,0.393836,0.715762,0.429978,0.143560,0.064039,0.225627,0.181818,0.237258,0.282051,0.10,0.285714,0.273504,0.270718,0.197368,0.219626,0.524390,0.441748,0.344262,0.387097,0.191781,1,2,0,6,15.4
2055,Alexis Ajinça,Luke Babbitt,139,2015,0,0.599315,0.668435,0.000000,0.234362,0.610837,0.582173,0.669421,0.148008,0.250000,0.40,0.386905,0.472826,0.287293,0.207317,0.247664,0.573265,0.306452,0.486239,0.411483,0.206107,3,14,13,0,6,0.297945,0.764858,0.655361,0.081214,0.078818,0.261838,0.210744,0.068541,0.205128,0.10,0.264881,0.165242,0.243094,0.131579,0.172897,0.431707,0.398058,0.270492,0.314516,0.157534,2,3,0,1,-1.7
2056,Matt Bonner,Patty Mills,196,2015,12,0.263699,0.511936,0.599562,0.117598,0.177340,0.169916,0.185950,0.127135,0.090909,0.10,0.050595,0.239130,0.254144,0.170732,0.196262,0.447301,0.322581,0.366972,0.358852,0.190840,2,1,0,3,0,0.363014,0.395349,0.592998,0.097621,0.147783,0.094708,0.115702,0.289982,0.384615,0.02,0.190476,0.441595,0.215470,0.184211,0.163551,0.419512,0.446602,0.237705,0.342742,0.171233,3,13,18,3,-2.0


In [10]:
embeddings_input = []
emb_dict = dict(cat_features_emb)
for k,v in encoding.items():
    embeddings_input.append((k, len(v), emb_dict[k]))
deep_column_idx = {k:v for v,k in enumerate(train_fin.columns)}

In [14]:
model_dd = DeepDense(embeddings_input = embeddings_input,
                   embeddings_encoding_dict = emb_dict,
                   continuous_cols = cont_features,
                   deep_column_idx = deep_column_idx,
                   hidden_layers = [150,100,25],
                   dropout = [0.2, 0.1],
                   output_dim = 1
                  )

## 150, 100, 25
## 0.2, 0.1
## Step 0.025, 0.05
model_dd.model_compile(method='regression', optimizer=['Adam', 0.025], lr_scheduler=['StepLR', 3, 0.05]) ## try cosine annealing
use_cuda = torch.cuda.is_available()
if use_cuda:
    model1 = model1.cuda()
use_cuda

228
[228, 150, 100, 25]


False

In [15]:
traindataset = model_dataloader(train_fin.values, train_y, mode = 'train', uhlog = False)
validdataset = model_dataloader(valid_fin.values, valid_y, mode = 'train', uhlog = False)
testdataset = model_dataloader(test_fin.values, test_y, mode = 'test', uhlog = False)

train_loader = torch.utils.data.DataLoader(dataset=traindataset, batch_size=128,shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=validdataset, batch_size=128,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testdataset, batch_size=128,shuffle=False)

model_dd.fit(n_epochs=10, train_loader=train_loader, eval_loader=valid_loader)

valid: 100%|██████████| 10/10 [00:00<00:00, 66.57it/s, loss=5.29]


In [18]:
from sklearn.metrics import mean_squared_error, r2_score

preds = model_dd.predict(test_loader)
print('RMSE')
print(np.sqrt(mean_squared_error(test_y, preds)))
print('\nMAE')
print(mean_absolute_error(test_y, preds))
print('\nR2')
print(r2_score(test_y, preds))

predict: 100%|██████████| 10/10 [00:00<00:00, 119.84it/s]

RMSE
5.306666202473572

MAE
4.094494451750751

R2
0.5208792368228222


In [19]:
from sklearn.ensemble import GradientBoostingRegressor

clf = GradientBoostingRegressor(loss='ls', 
                                n_estimators=100,
                                verbose = 1,
                               validation_fraction = 0.1)

clf.fit(train_fin, train_y)
clfpreds = clf.predict(test_fin)

print('RMSE')
print(np.sqrt(mean_squared_error(test_y, clfpreds)))
print('\nMAE')
print(mean_absolute_error(test_y, clfpreds))
print('\nR2')
print(r2_score(test_y, clfpreds))

      Iter       Train Loss   Remaining Time 
         1          53.7438            5.38s
         2          51.1481            4.22s
         3          48.9715            3.72s
         4          47.1510            3.60s
         5          45.5834            3.63s
         6          44.2222            3.62s
         7          43.0594            3.60s
         8          42.0670            3.58s
         9          41.1672            3.56s
        10          40.3873            3.51s
        20          35.6576            2.83s
        30          33.2115            2.23s
        40          31.8414            1.80s
        50          30.8269            1.50s
        60          30.0037            1.14s
        70          29.3283            0.82s
        80          28.7482            0.54s
        90          28.2631            0.26s
       100          27.8153            0.00s
RMSE
5.514728937817262

MAE
4.24446682975311

R2
0.4825721602614611


In [20]:
testplayers = testplayers.reset_index(drop = True)
testplayers['actual'] = test_y
testplayers['prediction'] = preds
testplayers['delta'] = test_y - preds
print(max(preds))
print(min(preds))
testplayers.sort_values(by='prediction', ascending = False).head(n=50)

17.499702
-22.989243


,player_left,player_right,team_year,actual,prediction,delta
107,Stephen Curry,Klay Thompson,GSW_2016,19.5,17.499702,2.000298
18,Stephen Curry,Kevin Durant,GSW_2017,19.3,17.454540,1.845460
112,Tim Duncan,Kawhi Leonard,SAS_2016,15.3,16.793633,-1.493633
817,Kevin Durant,Russell Westbrook,OKC_2013,10.8,16.154055,-5.354055
1130,Stephen Curry,Draymond Green,GSW_2015,17.7,15.905674,1.794326
128,Boban Marjanović,Patty Mills,SAS_2016,9.1,15.514173,-6.414173
121,Danny Green,Kawhi Leonard,SAS_2016,13.1,15.072808,-1.972808
109,Harrison Barnes,Stephen Curry,GSW_2016,16.3,15.051545,1.248455
466,James Harden,P.J. Tucker,HOU_2018,8.9,13.901670,-5.001670
453,Trevor Ariza,Chris Paul,HOU_2018,11.3,13.707647,-2.407647


In [28]:
cat_features

['team_year',
 'age_left',
 'position_left',
 'height_left',
 'country_left',
 'yrpro_left',
 'age_right',
 'position_right',
 'height_right',
 'country_right',
 'yrpro_right']

In [83]:
def extract_embeddings(model, test_df_detail, test_df_transform, cat_features, cont_features, playername1, playername2, yr):
    
    test_df_detail = test_df_detail.reset_index(drop = True)
    feat_list = [feat for sl in [cat_features, cont_features] for feat in sl]
    dfnew = pd.concat([test_df_detail[['player_left', 'player_right', 'year']], test_df_transform[feat_list]], axis = 1)
    #print(playername1)
    #print(playername2)
    dfnewfilt = dfnew[(dfnew['year'] == yr) & (dfnew['player_left'] == playername1) & (dfnew['player_right'] == playername2)]
    #cat_feat_use = [feat for feat in cat_features if '_left' in feat]
    #cont_feat_use = [feat for feat in cont_features if '_left' in feat]
                
    if len(dfnewfilt) < 1:
        dfnewfilt = dfnew[(dfnew['year'] == yr) & (dfnew['player_right'] == playername1) & dfnew['player_left'] == playername2]
        #cat_feat_use = [feat for feat in cat_features if '_right' in feat]
        #cont_feat_use = [feat for feat in cont_features if '_right' in feat]
        if len(dfnewfilt) < 1:
            #print('That player-year combination is not in the database')
            return
    if len(dfnewfilt) > 1:
        dfnewfilt = dfnewfilt.head(n=1)
        
    layer_details = list(model.named_parameters())
    
    embed_layers = [layer for layer in layer_details if layer[0] in ['emb_layer_position_left.weight',
                                                                     'emb_layer_height_left.weight', 
                                                                     'emb_layer_position_right.weight',
                                                                     'emb_layer_height_right.weight']]
    
    emb_vals = []
    
    cat_sim_feats = ['position_left', 'height_left', 'position_right', 'height_right']
    
    for feat in range(1,len(cat_sim_feats), 1):
        
        ## Something getting messed up in embedding extraction
        ## Feature weights not lining up.
    
        layer_name = embed_layers[feat][0]
        layer_weights = embed_layers[feat][1]
        emb_input = torch.LongTensor(dfnewfilt[cat_sim_feats[feat]].values)
    
        embed_output = nn.Embedding.from_pretrained(layer_weights)
        weights = embed_output(emb_input).numpy().tolist()
        weights = [weight for sl in weights for weight in sl]
        
        emb_vals.append(weights)
        
    emb_vals = [embed for sl in emb_vals for embed in sl]
    cont_output = np.squeeze(dfnewfilt[cont_features].values.tolist()) ## Experiment w/ different combos
    emb_concat = list([emb_vals, cont_output])
    emb_sl = [embed for sl in emb_concat for embed in sl]
        
    return emb_sl
    
    
emb_vals = extract_embeddings(model_dd, test, test_fin, cat_features, cont_features, 'Al Horford', 'Kyrie Irving', 2019)

In [41]:
## Get all unique player/team_year combinations

unique_players = modeling_dataset[['player_left', 'player_right', 'year']].values.tolist()
#unique_players['left_year'] = unique_players['player_left'] + '_' + unique_players['year'].map(str)
#unique_players['right_year'] = unique_players['player_left'] + '_' + unique_players['year'].map(str)

#uniqueset = [unique_players['left_year'].values.tolist(), unique_players['right_year'].values.tolist()]
#uniqueplayers = list(set([player for sl in uniqueset for player in sl]))

embed_db = []
for val in np.arange(len(unique_players)):
    
    player1 = unique_players[val][0]
    player2 = unique_players[val][1]
    year = int(unique_players[val][2])
    
    embeddingnew = extract_embeddings(model_dd, train, train_fin, cat_features, cont_features, player1, player2, year)
    if embeddingnew is None:
        embeddingnew = extract_embeddings(model_dd, valid, valid_fin, cat_features, cont_features, player1, player2, year)
        if embeddingnew is None:
            embeddingnew = extract_embeddings(model_dd, test, test_fin, cat_features, cont_features, player1, player2, year)
    
    embed_db.append([player1, player2, year, embeddingnew])
    


In [78]:
pred_example = train[(train['player_left'] == 'LeBron James') & (train['year'] == 2016) | (train['player_right'] == 'Stephen Curry') & (train['year'] == 2016)]
lebron = pred_example[pred_example['player_left'] == 'LeBron James'].head(n=1)
lebron.reset_index(drop=True, inplace = True)
lebron = lebron[['player_left', 'age_left', 'per_left', 'ts_left', '3par_left', 'ftr_left',
                                  'orb_left', 'drb_left', 'trb_left', 'ast_left', 'stl_left', 'blk_left', 'tov_left',
                                  'usg_left', 'ows_left', 'dws_left', 'ws_left', 'ws48_left', 'obpm_left', 'dbpm_left',
                                  'bpm_left', 'vorp_left', 'position_left', 'height_left',
                                  'country_left', 'yrpro_left']]
steph = pred_example[pred_example['player_right'] == 'Stephen Curry'].head(n=1)
steph.reset_index(drop=True, inplace = True)
steph = steph[['player_right', 'team_year', 'year', 'age_right', 'per_right', 'ts_right', '3par_right', 'ftr_right',
                                  'orb_right', 'drb_right', 'trb_right', 'ast_right', 'stl_right', 'blk_right', 'tov_right',
                                  'usg_right', 'ows_right', 'dws_right', 'ws_right', 'ws48_right', 'obpm_right', 'dbpm_right',
                                  'bpm_right', 'vorp_right', 'position_right', 'height_right', 
                                  'country_right', 'yrpro_right', 'pts']]

stephlebronraw = pd.concat([lebron, steph], axis=1)
stephlebron = stephlebronraw[['team_year', 'age_left', 'per_left', 'ts_left', '3par_left', 'ftr_left', 'orb_left',
                          'drb_left', 'trb_left', 'ast_left', 'stl_left', 'blk_left', 'tov_left', 'usg_left', 'ows_left',
                          'dws_left', 'ws_left', 'ws48_left', 'obpm_left', 'dbpm_left', 'bpm_left', 'vorp_left',
                          'position_left', 'height_left', 'country_left', 'yrpro_left',
                          'age_right', 'per_right', 'ts_right', '3par_right', 'ftr_right', 'orb_right',
                          'drb_right', 'trb_right', 'ast_right', 'stl_right', 'blk_right', 'tov_right', 'usg_right', 'ows_right',
                          'dws_right', 'ws_right', 'ws48_right', 'obpm_right', 'dbpm_right', 'bpm_right', 'vorp_right',
                          'position_right', 'height_right', 'country_right', 'yrpro_right']].copy()

sldataset = model_dataloader(stephlebron.values, test_y[0], mode = 'test', uhlog = False)
sl_loader = torch.utils.data.DataLoader(dataset=sldataset, batch_size=128,shuffle=False)

In [81]:
sl_comp = extract_embeddings(model_dd, stephlebronraw, stephlebron, cat_features, cont_features, 'LeBron James', 'Stephen Curry', 2016)
euc_distance = []
for val in np.arange(len(embed_db)):
        
    #sim = 1-spatial.distance.cosine(playerinfo, playercomps[val][1])
    sim = np.linalg.norm(np.array(sl_comp) - np.array(embed_db[val][3]))
        
    euc_distance.append([embed_db[val][0], embed_db[val][1], embed_db[val][2], sim])

slpred = model_dd.predict(sl_loader)

## Including height + position categorical features

print("Lebron & Steph's expected Net Rating on the 2016 Warriors is: " + str(slpred))

print('\nThe Closest historic comparisons to this two-player combination are:')

sorted(euc_distance, key=lambda x: x[3])[0:10]

predict: 100%|██████████| 1/1 [00:00<00:00, 213.91it/s]

Lebron & Steph's expected Net Rating on the 2016 Warriors is: [19.560759]

The Closest historic comparisons to this two-player combination are:


[['Harrison Barnes', 'Stephen Curry', 2015, 1.3229111349555864],
 ['Harrison Barnes', 'Stephen Curry', 2016, 1.4264843619621637],
 ['Gordon Hayward', 'George Hill', 2017, 1.536553288172524],
 ['Carmelo Anthony', 'Chauncey Billups', 2010, 1.5483834792473399],
 ['Gordon Hayward', 'Kyrie Irving', 2019, 1.5609445113288163],
 ['Nicolas Batum', 'Damian Lillard', 2015, 1.622644214213128],
 ['Nicolas Batum', 'Damian Lillard', 2014, 1.6227214865170574],
 ['Paul Millsap', 'Deron Williams', 2010, 1.6665267118752447],
 ['Harrison Barnes', 'Stephen Curry', 2014, 1.7423135624366766],
 ['Harrison Barnes', 'Stephen Curry', 2013, 1.8581554970879788]]

In [243]:
len(euc_distance)

2723

## To Do List

1. Confirm similarity working as intended - why such a big difference when switching left & right? Normalize cat embeddings before euc comp? Use cosine comp?

2. Create embed db of just continuous fatures for comparison's sake


In [136]:
## WIP - one player not current approach

from scipy import spatial

print(len(embed_db))
#print(embed_db[0])

## Next - Euclidian Distance

def player_comps(db, player, year):
    
    player_year = player + '_' + str(year)
    print(player_year)
    
    playername = [plyr[0] for plyr in db if plyr[0] == player_year]
    playerinfo = [plyr[1] for plyr in db if plyr[0] == player_year]
    
    playercomps = [plyr for plyr in db if plyr[0] != player_year]
    
    euc_distance = []
    for val in np.arange(len(playercomps)):
        
        #sim = 1-spatial.distance.cosine(playerinfo, playercomps[val][1])
        sim = np.linalg.norm(np.array(playerinfo) - np.array(playercomps[val][1]))
        
        euc_distance.append([playercomps[val][0], sim])
    
    print('The Closest Comparisons to ' + player + ' in ' + str(year) + ' are:')
        
    return euc_distance
    
sorted_players = player_comps(embed_db, 'Stephen Curry', 2016)

sorted(sorted_players, key=lambda x: x[1])[0:10]

## Run 10 different models w/ different datasets for both DL & RF & track results

2723
Stephen Curry_2016
The Closest Comparisons to Stephen Curry in 2016 are:


[['Stephen Curry_2015', 0.40534896737067605],
 ['James Harden_2019', 0.48136544018515626],
 ['James Harden_2018', 0.5141813443378047],
 ['LeBron James_2013', 0.6315734641685403],
 ['James Harden_2015', 0.6317141342215336],
 ['Kevin Durant_2014', 0.6385270321306987],
 ['LeBron James_2010', 0.6386624782660983],
 ['LeBron James_2014', 0.6434741383267008],
 ['James Harden_2017', 0.7371976451868181],
 ['Stephen Curry_2017', 0.7405318616997429]]

In [ ]:
def ds_creator(df, year):
    
    ds = df[df['year'] == year]
    
    return ds

ds_2010 = ds_creator(dataset_joiner, '2010')
ds_2011 = ds_creator(dataset_joiner, '2011')
ds_2012 = ds_creator(dataset_joiner, '2012')
ds_2013 = ds_creator(dataset_joiner, '2013')
ds_2014 = ds_creator(dataset_joiner, '2014')
ds_2015 = ds_creator(dataset_joiner, '2015')
ds_2016 = ds_creator(dataset_joiner, '2016')
ds_2017 = ds_creator(dataset_joiner, '2017')
ds_2018 = ds_creator(dataset_joiner, '2018')
ds_2019 = ds_creator(dataset_joiner, '2019')

ds_2010.to_csv('/Users/Wesley/bball/dataset_2010.csv', index = False)
ds_2011.to_csv('/Users/Wesley/bball/dataset_2011.csv', index = False)
ds_2012.to_csv('/Users/Wesley/bball/dataset_2012.csv', index = False)
ds_2013.to_csv('/Users/Wesley/bball/dataset_2013.csv', index = False)
ds_2014.to_csv('/Users/Wesley/bball/dataset_2014.csv', index = False)
ds_2015.to_csv('/Users/Wesley/bball/dataset_2015.csv', index = False)
ds_2016.to_csv('/Users/Wesley/bball/dataset_2016.csv', index = False)
ds_2017.to_csv('/Users/Wesley/bball/dataset_2017.csv', index = False)
ds_2018.to_csv('/Users/Wesley/bball/dataset_2018.csv', index = False)
ds_2019.to_csv('/Users/Wesley/bball/dataset_2019.csv', index = False)

In [61]:
from sklearn.metrics import mean_squared_error

print(np.sqrt(mean_squared_error(test_y, y_pred)))

5.4370715579342255


In [62]:
feature_importance = clf.feature_importances_ 
feats = list(zip(feature_importance, train_fin.columns))
feats.sort(key = lambda x: -x[0]) 

In [63]:
print(feats)

[(0.19750437868326715, 'ws48_left'), (0.15927061009869425, 'ws48_right'), (0.027964460914878867, 'dws_right'), (0.021628296779794796, '3par_left'), (0.02089002995993765, 'per_left'), (0.020583765696687163, 'ftr_left'), (0.019148407609345438, 'tov_left'), (0.01866294665942738, 'dws_left'), (0.018662042811026416, 'ts_left'), (0.018602723492973046, 'ftr_right'), (0.01685928601327464, 'team_year'), (0.016735220164907375, 'per_right'), (0.01668042100339041, 'usg_right'), (0.015791317260294757, '3par_right'), (0.015626249789592424, 'drb_right'), (0.015094990328599596, 'ts_right'), (0.014582517370563694, 'orb_left'), (0.014313614588086112, 'bpm_left'), (0.014183997047289872, 'drb_left'), (0.01412297962750411, 'usg_left'), (0.014092345876286395, 'ast_right'), (0.014036018235878352, 'tov_right'), (0.013873488982498287, 'orb_right'), (0.013646493464981578, 'ast_left'), (0.012770515809658822, 'ws_right'), (0.012767801788834418, 'ws_left'), (0.012641595675613822, 'vorp_right'), (0.0121269101512598